<a href="https://colab.research.google.com/github/chenboju/AI/blob/main/0504_Multiple_sequences_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling multiple sequences (PyTorch)
* 如何處理多個序列？
* 如何處理不同長度的多個序列？
* 詞彙索引是模型正常運作的唯一輸入嗎？
* 是否存在序列太長之類的情況？

安裝套件

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


將序列轉換為數字list。將此數字list轉換為張量並將其發送到模型

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

IndexError: too many indices for tensor of dimension 1

In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


上例向模型發送了單個序列，而Transformers 模型預設需要多個句子。此例分詞器應用到sequence，不僅將輸入 ID 列表轉換為張量，而且還在其上添加了一個維度(加上`[]`)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


### padding
向模型提供多個句子時，稱為batch(批次)，批次允許模型發揮作用。使用多個序列就像使，但是將兩個（或更多）句子批次在一起時，它們的長度可能不同。模型只吃是矩形形狀的張量，因此您無法將輸入 ID 清單直接轉換為張量。為了解決這個問題，我們通常會填入一個「填充值」稱為padding。

下面ID無法輸入模型

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200]
]

為了解決這個問題，我們將使用padding使張量具有矩形形狀。填充透過在值較少的句子上添加一個稱為「填充標記」的特殊單字來確保所有句子具有相同的長度。如果有 10 個包含 10 個單字的句子和 1 個包含 20 個單字的句子，填充將確保所有句子都有 20 個單字。範例中產生的張量如下所示：

In [ ]:
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

### padding token
padding的token ID 必須在預訓練時就先定義好，在tokenizer的設定檔中會指定這個token。下例使用並通過模型單獨發送兩個句子並一起批處理

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


### 一模一樣的文字值不同
批量預測中的邏輯有問題：第二行應該與第二句話的邏輯相同，但我們得到了完全不同的值，這是因為 Transformer 模型的關鍵特徵是將每個token置於上下文中的注意力層。會考慮到padding token也被拿去做注意力機制，但padding不應該參加注意力機制。為了在通過模型傳遞不同長度的單一句子或傳遞具有相同句子，並應用padding的批次時獲得相同的結果，我們需要告訴這些注意層忽略padding。這是透過使用`attention_mask`來完成的。

`attention_mask`是與輸入 ID 張量形狀完全相同的張量，用 0 和 1 填充：1 表示應注意相應的token，0 表示不應注意相應的token（即，應忽略它們）模型的注意力層）。

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


### 更長的句子
對於 Transformer 模型，模型的處理序列長度是有限的。大多數模型可處理最多 512 或 1024 個標記的序列，並且當要求處理更長的序列時會當機。這個問題有兩種解決方案：

* 使用支援的序列長度較長的模型。
* 截斷序列。

不同模型具有不同的支援序列長度，有些模型專門處理非常長的序列。Longformer是一個例子，另一個例子是LED。否則，我們建議您透過指定參數來截斷序列`max_sequence_length`：

In [ ]:
sequence = sequence[:max_sequence_length]

NameError: name 'max_sequence_length' is not defined